In [9]:
import csv
import json
import requests
from lxml import etree


class DataScraper:
    def __init__(self):
        self.pagename_type = {
            "业绩报表": "RPT_LICO_FN_CPD",
            "业绩快报": "RPT_FCI_PERFORMANCEE",
            "业绩预告": "RPT_PUBLIC_OP_NEWPREDICT",
            "预披露时间": "RPT_PUBLIC_BS_APPOIN",
            "资产负债表": "RPT_DMSK_FN_BALANCE",
            "利润表": "RPT_DMSK_FN_INCOME",
            "现金流量表": "RPT_DMSK_FN_CASHFLOW"
        }

        self.pagename_en = {
            "业绩报表": "yjbb",
            "业绩快报": "yjkb",
            "业绩预告": "yjyg",
            "预约披露时间": "yysj",
            "资产负债表": "zcfz",
            "利润表": "lrb",
            "现金流量表": "xjll"
        }

        self.en_list = []

        self.url = 'https://datacenter-web.eastmoney.com/api/data/v1/get'
        self.headers = {
            'Accept': '*/*',
            'Accept-Language': 'zh-CN,zh;q=0.9',
            'Connection': 'closed',
            'Referer': 'https://data.eastmoney.com/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
            'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"'
        }

    def get_table(self, page):
        params = {
            'sortTypes': '-1,-1',
            'reportName': self.table_type,
            'columns': 'ALL',
            'filter': f'(REPORT_DATE=\'{self.timePoint}\')'
        }

        if self.table_type in ['RPT_LICO_FN_CPD']:
            params['filter'] = f'(REPORTDATE=\'{self.timePoint}\')'
        params['pageNumber'] = str(page)
        response = requests.get(url=self.url, params=params, headers=self.headers)
        data = json.loads(response.text)
        if data['result']:
            return data['result']['data']
        else:
            return

    def get_header(self, all_en_list):
        ch_list = []
        url = f'https://data.eastmoney.com/bbsj/{self.pagename_en[self.pagename]}.html'
        response = requests.get(url)
        res = etree.HTML(response.text)
        for en in all_en_list:
            ch = ''.join(
                [i.strip() for i in res.xpath(f'//div[@class="dataview"]//table[1]//th[@data-field="{en}"]//text()')])
            if ch:
                ch_list.append(ch)
                self.en_list.append(en)
        return ch_list

    def write_header(self, table_data):
        with open(self.filename, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            headers = self.get_header(list(table_data[0].keys()))
            writer.writerow(headers)

    def write_table(self, table_data):
        with open(self.filename, 'a', encoding='utf-8', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for item in table_data:
                row = []
                for key in item.keys():
                    if key in self.en_list:
                        row.append(str(item[key]))
                print(row)
                writer.writerow(row)

    def get_timeList(self):
        headers = {
            'Referer': 'https://data.eastmoney.com/bbsj/202206.html',
        }
        response = requests.get('https://data.eastmoney.com/bbsj/202206.html', headers=headers)
        res = etree.HTML(response.text)
        return res.xpath('//*[@id="filter_date"]//option/text()')

    def run(self):
        self.timeList = self.get_timeList()
        for index, value in enumerate(self.timeList):
            if (index + 1) % 5 == 0:
                print(value)
            else:
                print(value, end=' ; ')

        self.timePoint = str(input('\n请选择时间（可选项如上）:'))
        self.pagename = str(
            input('请输入报表类型（业绩报表;业绩快报;业绩预告;预约披露时间;资产负债表;利润表；现金流量表）:'))
        assert self.timePoint in self.timeList, '时间输入错误'
        assert self.pagename in list(self.pagename_type.keys()), '报表类型输入错误'
        self.table_type = self.pagename_type[self.pagename]
        self.filename = f'{self.pagename}_{self.timePoint}.csv'
        self.write_header(self.get_table(1))
        page = 1
        while True:
            table = self.get_table(page)
            if table:
                self.write_table(table)
            else:
                break
            page += 1


if __name__ == '__main__':
    scraper = DataScraper()
    scraper.run()

2024-12-31 ; 2024-09-30 ; 2024-06-30 ; 2024-03-31 ; 2023-12-31
2023-09-30 ; 2023-06-30 ; 2023-03-31 ; 2022-12-31 ; 2022-09-30
2022-06-30 ; 2022-03-31 ; 2021-12-31 ; 2021-09-30 ; 2021-06-30
2021-03-31 ; 2020-12-31 ; 2020-09-30 ; 2020-06-30 ; 2020-03-31
2019-12-31 ; 2019-09-30 ; 2019-06-30 ; 2019-03-31 ; 2018-12-31
2018-09-30 ; 2018-06-30 ; 2018-03-31 ; 2017-12-31 ; 2017-09-30
2017-06-30 ; 2017-03-31 ; 2016-12-31 ; 2016-09-30 ; 2016-06-30
2016-03-31 ; 2015-12-31 ; 2015-09-30 ; 2015-06-30 ; 2015-03-31
2014-12-31 ; 2014-09-30 ; 2014-06-30 ; 2014-03-31 ; 2013-12-31
2013-09-30 ; 2013-06-30 ; 2013-03-31 ; 2012-12-31 ; 2012-09-30
2012-06-30 ; 2012-03-31 ; ['000001', '平安银行', '2015-04-24 00:00:00', '0.37', '16100000000', '5054000000', '4.41', '49.0464728754', '40.8191696851', '12.320463568531', '4.38', 'None', '8.4613', '42.9703', 'None', '银行']
['000002', '万科A', '2015-04-27 00:00:00', '0.139', '9497216581.95', '1529479377.61', '1.97', '-32.1622826183', '-5.2310942148', '6.676704808833', '-0.51',